# Glassdoor Data Cleaning

In [57]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unidecode import unidecode
from langdetect import detect
import pycountry_convert as pc
from nltk.corpus import stopwords
import collections
%matplotlib inline

### Read in raw file

In [7]:
file = open('./glassdoor_data/columns.txt', "r")
columns = [line.strip() for line in file]
file.close()
print(list(columns))

['gaTrackerData.category', 'gaTrackerData.empId', 'gaTrackerData.empName', 'gaTrackerData.empSize', 'gaTrackerData.industry', 'gaTrackerData.industryId', 'gaTrackerData.jobTitle', 'gaTrackerData.location', 'gaTrackerData.locationId', 'gaTrackerData.locationType', 'gaTrackerData.sector', 'gaTrackerData.sectorId', 'header.employerId', 'header.employerName', 'header.gocId', 'header.jobTitle', 'header.locId', 'header.location', 'header.locationType', 'header.posted', 'header.rating', 'job.description', 'job.jobTitleId', 'map.country', 'map.employerName', 'map.lat', 'map.lng', 'map.location', 'overview.industry', 'overview.industryId', 'overview.revenue', 'overview.sector', 'overview.sectorId', 'overview.size', 'overview.type', 'rating.starRating', 'salary.salaries']


In [8]:
df = pd.read_csv('./glassdoor_data/glassdoor.csv')

In [9]:
df.head()

,benefits.benefitRatingDecimal,benefits.comments,benefits.highlights,benefits.numRatings,benefits.employerSummary,breadCrumbs,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,...,salary.currency.displayName,salary.currency.id,salary.currency.name,salary.currency.negativeTemplate,salary.currency.new,salary.currency.positiveTemplate,salary.currency.symbol,salary.lastSalaryDate,salary.salaries,wwfu
0,0.0,2801.0,2801.0,0,NaN,2801,-1,2183810,Modus Group,501-1000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2483.0,NaN
1,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-14T06:05:58,2484.0,NaN
2,0.0,2803.0,2803.0,0,NaN,2803,-1,1043373,Immobiliare.it,201-500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2485.0,NaN
3,0.0,2804.0,2804.0,0,NaN,2804,-1,2226886,Codup,1-50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2486.0,NaN
4,0.0,2805.0,2805.0,0,NaN,2805,-1,851555,Markelytics Solutions,201-500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2487.0,NaN


In [10]:
df_g = df[columns]

In [11]:
df_g.head()

,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,gaTrackerData.industry,gaTrackerData.industryId,gaTrackerData.jobTitle,gaTrackerData.location,gaTrackerData.locationId,gaTrackerData.locationType,...,map.location,overview.industry,overview.industryId,overview.revenue,overview.sector,overview.sectorId,overview.size,overview.type,rating.starRating,salary.salaries
0,-1,2183810,Modus Group,501-1000,Investment Banking & Asset Management,200053,Biogas Project Development Manager,Warsaw,3094484,CITY,...,Warsaw,Investment Banking & Asset Management,200053,Unknown / Non-Applicable,Finance,10010,501 to 1000 employees,Company - Private,-0.1,2483.0
1,-1,10416,Boehringer Ingelheim,10000--1,Biotech & Pharmaceuticals,200021,Quality Manager - Boehringer Ingelheim Healthc...,NaN,3183562,CITY,...,Porto,Biotech & Pharmaceuticals,200021,£5 to £10 billion (GBP),Biotech & Pharmaceuticals,10005,10000+ employees,Company - Private,3.8,2484.0
2,-1,1043373,Immobiliare.it,201-500,NaN,0,"Senior Software Engineer (PHP, Elixir, Python)",Milano,2802090,CITY,...,Milan,NaN,0,£25 to £50 million (GBP),NaN,0,201 to 500 employees,Company - Private,2.4,2485.0
3,-1,2226886,Codup,1-50,NaN,0,Senior SQA Engineer,Karachi,3212296,CITY,...,Karāchi,NaN,0,Unknown / Non-Applicable,NaN,0,1 to 50 employees,Company - Private,1.5,2486.0
4,-1,851555,Markelytics Solutions,201-500,Consulting,200028,Research Manager,Bengaluru,2940587,CITY,...,Bengaluru,Consulting,200028,Unknown / Non-Applicable,Business Services,10006,201 to 500 employees,Company - Private,3.5,2487.0


In [12]:
df_g.isnull().sum().sort_values(ascending = False)

gaTrackerData.location        70283
map.country                   50028
gaTrackerData.industry        47405
overview.industry             47405
overview.sector               47373
gaTrackerData.sector          47373
rating.starRating             18391
gaTrackerData.empName         18209
gaTrackerData.empSize         18209
overview.type                 18209
overview.size                 18209
overview.revenue              18209
salary.salaries               18141
map.employerName              18141
header.employerName            3245
header.locationType              23
header.location                   5
map.location                      5
job.description                   4
header.locId                      0
header.employerId                 0
gaTrackerData.industryId          0
gaTrackerData.jobTitle            0
gaTrackerData.locationId          0
overview.sectorId                 0
gaTrackerData.locationType        0
gaTrackerData.sectorId            0
overview.industryId         

### Clean up country names 

In [13]:
# get country codes
country_codes = pd.read_csv("./glassdoor_data/country_names_2_digit_codes.csv")

In [14]:
country_codes.head()

,Name,Code
0,Afghanistan,AF
1,Ã…land Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [15]:
# drop nans for country data
listings_before = df_g.shape[0]
print(f'Initial Glassdoor dataset length is {listings_before} job listings.')

df_g.dropna(subset=['map.country'], inplace=True)

listings_after = df_g.shape[0]
print(f'After removing NaN countries we were left with {listings_after} job listings.')

Initial Glassdoor dataset length is 165290 job listings.
After removing NaN countries we were left with 115262 job listings.


<ipython-input-15-5c9a97b82ee8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g.dropna(subset=['map.country'], inplace=True)


In [16]:
df_g = pd.merge(df_g, country_codes, left_on='map.country', right_on='Code', how='left')
df_g.head()

,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,gaTrackerData.industry,gaTrackerData.industryId,gaTrackerData.jobTitle,gaTrackerData.location,gaTrackerData.locationId,gaTrackerData.locationType,...,overview.industryId,overview.revenue,overview.sector,overview.sectorId,overview.size,overview.type,rating.starRating,salary.salaries,Name,Code
0,-1,2183810,Modus Group,501-1000,Investment Banking & Asset Management,200053,Biogas Project Development Manager,Warsaw,3094484,CITY,...,200053,Unknown / Non-Applicable,Finance,10010,501 to 1000 employees,Company - Private,-0.1,2483.0,NaN,NaN
1,-1,1043373,Immobiliare.it,201-500,NaN,0,"Senior Software Engineer (PHP, Elixir, Python)",Milano,2802090,CITY,...,0,£25 to £50 million (GBP),NaN,0,201 to 500 employees,Company - Private,2.4,2485.0,NaN,NaN
2,-1,2226886,Codup,1-50,NaN,0,Senior SQA Engineer,Karachi,3212296,CITY,...,0,Unknown / Non-Applicable,NaN,0,1 to 50 employees,Company - Private,1.5,2486.0,Pakistan,PK
3,-1,851555,Markelytics Solutions,201-500,Consulting,200028,Research Manager,Bengaluru,2940587,CITY,...,200028,Unknown / Non-Applicable,Business Services,10006,201 to 500 employees,Company - Private,3.5,2487.0,India,IN
4,-1,556626,Nisha Group,51-200,Staffing & Outsourcing,200032,B2B Product Manager,NaN,-1,CITY,...,200032,Unknown / Non-Applicable,Business Services,10006,51 to 200 employees,Company - Private,3.5,2489.0,Israel,IL


In [17]:
df_g['map.country'] = df_g.Name.fillna(df_g['map.country'] )
df_g[['gaTrackerData.location', 'map.country', 'Name', 'Code']].head()
df_g.head()

,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,gaTrackerData.industry,gaTrackerData.industryId,gaTrackerData.jobTitle,gaTrackerData.location,gaTrackerData.locationId,gaTrackerData.locationType,...,overview.industryId,overview.revenue,overview.sector,overview.sectorId,overview.size,overview.type,rating.starRating,salary.salaries,Name,Code
0,-1,2183810,Modus Group,501-1000,Investment Banking & Asset Management,200053,Biogas Project Development Manager,Warsaw,3094484,CITY,...,200053,Unknown / Non-Applicable,Finance,10010,501 to 1000 employees,Company - Private,-0.1,2483.0,NaN,NaN
1,-1,1043373,Immobiliare.it,201-500,NaN,0,"Senior Software Engineer (PHP, Elixir, Python)",Milano,2802090,CITY,...,0,£25 to £50 million (GBP),NaN,0,201 to 500 employees,Company - Private,2.4,2485.0,NaN,NaN
2,-1,2226886,Codup,1-50,NaN,0,Senior SQA Engineer,Karachi,3212296,CITY,...,0,Unknown / Non-Applicable,NaN,0,1 to 50 employees,Company - Private,1.5,2486.0,Pakistan,PK
3,-1,851555,Markelytics Solutions,201-500,Consulting,200028,Research Manager,Bengaluru,2940587,CITY,...,200028,Unknown / Non-Applicable,Business Services,10006,201 to 500 employees,Company - Private,3.5,2487.0,India,IN
4,-1,556626,Nisha Group,51-200,Staffing & Outsourcing,200032,B2B Product Manager,NaN,-1,CITY,...,200032,Unknown / Non-Applicable,Business Services,10006,51 to 200 employees,Company - Private,3.5,2489.0,Israel,IL


In [18]:
df_g = df_g.drop(columns=['Name', 'Code'])
df_g[['gaTrackerData.location', 'map.country']].head()

,gaTrackerData.location,map.country
0,Warsaw,Poland
1,Milano,Italy
2,Karachi,Pakistan
3,Bengaluru,India
4,NaN,Israel


In [19]:
listings_before = df_g.shape[0]

# Now we merge two dataframes by Country and Name
df_g = pd.merge(df_g, country_codes, left_on='map.country', right_on='Name', how='left')
df_g.head()

,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,gaTrackerData.industry,gaTrackerData.industryId,gaTrackerData.jobTitle,gaTrackerData.location,gaTrackerData.locationId,gaTrackerData.locationType,...,overview.industryId,overview.revenue,overview.sector,overview.sectorId,overview.size,overview.type,rating.starRating,salary.salaries,Name,Code
0,-1,2183810,Modus Group,501-1000,Investment Banking & Asset Management,200053,Biogas Project Development Manager,Warsaw,3094484,CITY,...,200053,Unknown / Non-Applicable,Finance,10010,501 to 1000 employees,Company - Private,-0.1,2483.0,Poland,PL
1,-1,1043373,Immobiliare.it,201-500,NaN,0,"Senior Software Engineer (PHP, Elixir, Python)",Milano,2802090,CITY,...,0,£25 to £50 million (GBP),NaN,0,201 to 500 employees,Company - Private,2.4,2485.0,Italy,IT
2,-1,2226886,Codup,1-50,NaN,0,Senior SQA Engineer,Karachi,3212296,CITY,...,0,Unknown / Non-Applicable,NaN,0,1 to 50 employees,Company - Private,1.5,2486.0,Pakistan,PK
3,-1,851555,Markelytics Solutions,201-500,Consulting,200028,Research Manager,Bengaluru,2940587,CITY,...,200028,Unknown / Non-Applicable,Business Services,10006,201 to 500 employees,Company - Private,3.5,2487.0,India,IN
4,-1,556626,Nisha Group,51-200,Staffing & Outsourcing,200032,B2B Product Manager,NaN,-1,CITY,...,200032,Unknown / Non-Applicable,Business Services,10006,51 to 200 employees,Company - Private,3.5,2489.0,Israel,IL


In [20]:
# And remove rows were Name is NaN
df_g.dropna(subset=['Name'], inplace=True)
df_g = df_g.drop(['Name', 'Code'], axis=1)
listings_after =df_g.shape[0]
print(f'Remaining job listings: {listings_after}')

Remaining job listings: 109555


In [21]:
df_g.isnull().sum().sort_values(ascending = False)

gaTrackerData.industry        41410
overview.industry             41410
gaTrackerData.sector          41378
overview.sector               41378
gaTrackerData.location        21264
rating.starRating             15327
overview.revenue              15176
gaTrackerData.empName         15176
gaTrackerData.empSize         15176
overview.type                 15176
overview.size                 15176
map.employerName              15111
salary.salaries               15111
header.employerName            1027
overview.industryId               0
job.jobTitleId                    0
map.location                      0
map.lng                           0
map.lat                           0
overview.sectorId                 0
map.country                       0
gaTrackerData.category            0
job.description                   0
header.rating                     0
header.posted                     0
gaTrackerData.empId               0
header.location                   0
header.locId                

### Clean up job titles

In [22]:
def parse_title(title):
    title_lower = title.lower()
    
    title_list = ['data scientist', 'software engineer', 'data analyst', 'research scientist', 'business analyst',
                  'data engineer', 'statistician', 'dba', 'database engineer', 'machine learning engineer', 'applied scientist',
                 'business intelligence', 'product manager', 'sales']
    
    for val in title_list:
        if val in title_lower:
            return val.title()
   
    return title_lower.title()

In [23]:
df_g['Job Title'] = df_g['gaTrackerData.jobTitle'].apply(parse_title)

In [24]:
df_g['Job Title'].value_counts()

Software Engineer                                                       10827
Business Analyst                                                         6706
Product Manager                                                          5594
Data Scientist                                                           4625
Data Analyst                                                             4165
                                                                        ...  
Consultant : Business & Process Analyst, Project & Change Management        1
Information Engineer                                                        1
Senior Speech Recognition Scientist                                         1
Insurance Product - It Manager                                              1
Field Research Associate                                                    1
Name: Job Title, Length: 29942, dtype: int64

### Clean dataframe for Tableau Map

In [25]:
glassdoor = df_g[['Job Title', 'gaTrackerData.empName', 'gaTrackerData.empSize', 'gaTrackerData.industry', 'gaTrackerData.location', 'gaTrackerData.sector', 'header.jobTitle', 'map.country', 'overview.type', 'rating.starRating']]
glassdoor.rename(columns={'gaTrackerData.empName': 'Employer Name', 
                          'gaTrackerData.empSize': 'Employer Size',
                          'gaTrackerData.industry': 'Industry',
                          'gaTrackerData.location': 'Location',
                          'gaTrackerData.sector': 'Sector',
                          'header.jobTitle': 'Listing Job Title',
                          'map.country': 'Country',
                          'overview.type': 'Company Type',
                          'rating.starRating': 'Company Rating'
                         }, 
                            inplace=True)
glassdoor.head()

/Users/haleyxue/opt/anaconda3/envs/jobs_project/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Job Title,Employer Name,Employer Size,Industry,Location,Sector,Listing Job Title,Country,Company Type,Company Rating
0,Biogas Project Development Manager,Modus Group,501-1000,Investment Banking & Asset Management,Warsaw,Finance,Biogas Project Development Manager,Poland,Company - Private,-0.1
1,Software Engineer,Immobiliare.it,201-500,NaN,Milano,NaN,"Senior Software Engineer (PHP, Elixir, Python)",Italy,Company - Private,2.4
2,Senior Sqa Engineer,Codup,1-50,NaN,Karachi,NaN,Senior SQA Engineer,Pakistan,Company - Private,1.5
3,Research Manager,Markelytics Solutions,201-500,Consulting,Bengaluru,Business Services,Research Manager,India,Company - Private,3.5
4,Product Manager,Nisha Group,51-200,Staffing & Outsourcing,NaN,Business Services,B2B Product Manager,Israel,Company - Private,3.5


In [26]:
len(glassdoor)

109555

In [64]:
# save dataframe 
glassdoor.to_excel('./glassdoor_map_tableau.xlsx',engine='xlsxwriter')  

## Clean job descriptions

In [27]:
descriptions = df_g[['Job Title', 'gaTrackerData.empName', 'gaTrackerData.empSize', 'gaTrackerData.industry', 
                  'gaTrackerData.location', 'gaTrackerData.sector', 'header.jobTitle', 'map.country', 
                  'overview.type', 'rating.starRating', 'job.description']]

descriptions.rename(columns={'gaTrackerData.empName': 'Employer Name', 
                          'gaTrackerData.empSize': 'Employer Size',
                          'gaTrackerData.industry': 'Industry',
                          'gaTrackerData.location': 'Location',
                          'gaTrackerData.sector': 'Sector',
                          'header.jobTitle': 'Listing Job Title',
                          'map.country': 'Country',
                          'overview.type': 'Company Type',
                          'rating.starRating': 'Company Rating',
                          'job.description': 'Job Description'}, 
                            inplace=True)

descriptions.head()

/Users/haleyxue/opt/anaconda3/envs/jobs_project/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Job Title,Employer Name,Employer Size,Industry,Location,Sector,Listing Job Title,Country,Company Type,Company Rating,Job Description
0,Biogas Project Development Manager,Modus Group,501-1000,Investment Banking & Asset Management,Warsaw,Finance,Biogas Project Development Manager,Poland,Company - Private,-0.1,We are the drivers of the future who offer a g...
1,Software Engineer,Immobiliare.it,201-500,NaN,Milano,NaN,"Senior Software Engineer (PHP, Elixir, Python)",Italy,Company - Private,2.4,Dottori.it &egrave; il motore di ricerca che t...
2,Senior Sqa Engineer,Codup,1-50,NaN,Karachi,NaN,Senior SQA Engineer,Pakistan,Company - Private,1.5,Codup is looking for a Senior SQA Engineer for...
3,Research Manager,Markelytics Solutions,201-500,Consulting,Bengaluru,Business Services,Research Manager,India,Company - Private,3.5,<strong>Department:</strong> Research &amp; Op...
4,Product Manager,Nisha Group,51-200,Staffing & Outsourcing,NaN,Business Services,B2B Product Manager,Israel,Company - Private,3.5,<b>&#1514;&#1497;&#1488;&#1493;&#1512; &#1492;...


In [28]:
# get only jobs of interest

title_list = ['data scientist', 'software engineer', 'data analyst', 'research scientist', 'business analyst',
              'data engineer', 'statistician', 'dba', 'database engineer', 'machine learning engineer', 'applied scientist',
             'business intelligence', 'product manager', 'sales']

filter_list =[]

for t in title_list:
    filter_list.append(t.title())
    
skills = descriptions[descriptions['Job Title'].isin(filter_list)]


In [31]:
# Filter for English descriptions
def replace_html(text):
    re_expression = '(<\w+>|</\w+>|\\n|<\w+/>|&quot;|&egrave|http\S+|[-|0-9]|\*|,|;|&)'
    clean_text = re.sub(re_expression, ' ', text)
    ready_text = unidecode(clean_text.lower())
    return ready_text

In [32]:
skills['Job Description'] = skills['Job Description'].map(replace_html)
skills.head(5)

<ipython-input-32-be71c2764000>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills['Job Description'] = skills['Job Description'].map(replace_html)


,Job Title,Employer Name,Employer Size,Industry,Location,Sector,Listing Job Title,Country,Company Type,Company Rating,Job Description
1,Software Engineer,Immobiliare.it,201-500,NaN,Milano,NaN,"Senior Software Engineer (PHP, Elixir, Python)",Italy,Company - Private,2.4,dottori.it il motore di ricerca che ti aiut...
4,Product Manager,Nisha Group,51-200,Staffing & Outsourcing,NaN,Business Services,B2B Product Manager,Israel,Company - Private,3.5,# # # # # # #...
8,Product Manager,HSBC Global Technology (HSBC GLT),-1-0,Banks & Credit Unions,NaN,Finance,"Digital Product Manager, Digital Customer Onbo...",Hong Kong,Company - Private,3.8,some careers grow faster than others. if yo...
11,Data Analyst,Amaris,5001-10000,Consulting,Brussels,Business Services,Big Data Analyst,Belgium,Company - Private,3.0,amaris est une soci eacute t eacute ind eacut...
13,Business Analyst,Accenture,10000--1,Consulting,Paris,Business Services,Business Analyst IFRS 17 Implementation Paris F/H,France,Company - Public,3.7,business analyst paris f/h contrat : cdi ...


In [38]:
def get_language(text):
    try:
        lang = detect(text)
    except:
        lang = 'Error'
    return lang

In [39]:
skills['Language'] = skills['Job Description'].apply(get_language)
skills.head()

<ipython-input-39-62f56bd0f750>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills['Language'] = skills['Job Description'].apply(get_language)


,Job Title,Employer Name,Employer Size,Industry,Location,Sector,Listing Job Title,Country,Company Type,Company Rating,Job Description,Language
1,Software Engineer,Immobiliare.it,201-500,NaN,Milano,NaN,"Senior Software Engineer (PHP, Elixir, Python)",Italy,Company - Private,2.4,dottori.it il motore di ricerca che ti aiut...,it
4,Product Manager,Nisha Group,51-200,Staffing & Outsourcing,NaN,Business Services,B2B Product Manager,Israel,Company - Private,3.5,# # # # # # #...,fr
8,Product Manager,HSBC Global Technology (HSBC GLT),-1-0,Banks & Credit Unions,NaN,Finance,"Digital Product Manager, Digital Customer Onbo...",Hong Kong,Company - Private,3.8,some careers grow faster than others. if yo...,en
11,Data Analyst,Amaris,5001-10000,Consulting,Brussels,Business Services,Big Data Analyst,Belgium,Company - Private,3.0,amaris est une soci eacute t eacute ind eacut...,fr
13,Business Analyst,Accenture,10000--1,Consulting,Paris,Business Services,Business Analyst IFRS 17 Implementation Paris F/H,France,Company - Public,3.7,business analyst paris f/h contrat : cdi ...,fr


In [40]:
# save a copy 
skills.to_excel('./glassdoor_lang.xlsx',engine='xlsxwriter')  

In [42]:
english = skills[skills['Language']=='en']
english.drop(columns=['Language'], inplace=True)
english.head()

/Users/haleyxue/opt/anaconda3/envs/jobs_project/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Job Title,Employer Name,Employer Size,Industry,Location,Sector,Listing Job Title,Country,Company Type,Company Rating,Job Description
8,Product Manager,HSBC Global Technology (HSBC GLT),-1-0,Banks & Credit Unions,NaN,Finance,"Digital Product Manager, Digital Customer Onbo...",Hong Kong,Company - Private,3.8,some careers grow faster than others. if yo...
14,Data Scientist,Harnham,51-200,Staffing & Outsourcing,Charing Cross,Business Services,Data Scientist media,United Kingdom,Company - Private,4.1,data scientist media london pou...
16,Sales,Rentokil Initial,10000--1,Building & Personnel Services,Perai,Business Services,"Territory Manager (Outdoor Sales Consultant), ...",Malaysia,Company - Public,4.4,rentokil initial: rentokil initial is an ...
20,Data Scientist,Derbysoft,51-200,IT Services,Barcelona,Information Technology,Data Scientist,Spain,Company - Private,3.8,job title: data scientist business unit:...
21,Business Analyst,TrackTik,51-200,Computer Hardware & Software,Montréal,Information Technology,Business Analyst,Canada,Company - Private,4.2,about tracktik tracktik is a montreal bas...


In [47]:
# Get continent
english = pd.merge(english, country_codes, left_on='Country', right_on='Name', how='left')
english['Region'] = english['Code'] .apply(pc.country_alpha2_to_continent_code)
english.head()

,Job Title,Employer Name,Employer Size,Industry,Location,Sector,Listing Job Title,Country,Company Type,Company Rating,Job Description,Name,Code,Region
0,Product Manager,HSBC Global Technology (HSBC GLT),-1-0,Banks & Credit Unions,NaN,Finance,"Digital Product Manager, Digital Customer Onbo...",Hong Kong,Company - Private,3.8,some careers grow faster than others. if yo...,Hong Kong,HK,AS
1,Data Scientist,Harnham,51-200,Staffing & Outsourcing,Charing Cross,Business Services,Data Scientist media,United Kingdom,Company - Private,4.1,data scientist media london pou...,United Kingdom,GB,EU
2,Sales,Rentokil Initial,10000--1,Building & Personnel Services,Perai,Business Services,"Territory Manager (Outdoor Sales Consultant), ...",Malaysia,Company - Public,4.4,rentokil initial: rentokil initial is an ...,Malaysia,MY,AS
3,Data Scientist,Derbysoft,51-200,IT Services,Barcelona,Information Technology,Data Scientist,Spain,Company - Private,3.8,job title: data scientist business unit:...,Spain,ES,EU
4,Business Analyst,TrackTik,51-200,Computer Hardware & Software,Montréal,Information Technology,Business Analyst,Canada,Company - Private,4.2,about tracktik tracktik is a montreal bas...,Canada,CA,NA


In [49]:
english.drop(columns=['Name'], inplace=True)
english['Region'].unique()

array(['AS', 'EU', 'NA', 'AF', 'OC', 'SA'], dtype=object)

In [51]:
def get_cont_name(code): 
    cont_conv_dict = {'AF': 'Africa',
                  'NA': 'North America',
                  'OC': 'Oceania',
                  'AN': 'Antarctica',
                  'AS': 'Asia',
                  'EU': 'Europe',
                  'SA': 'South America'}
    return cont_conv_dict[code] 

english['Region'] = english['Region'].apply(get_cont_name)
english.head()

,Job Title,Employer Name,Employer Size,Industry,Location,Sector,Listing Job Title,Country,Company Type,Company Rating,Job Description,Code,Region
0,Product Manager,HSBC Global Technology (HSBC GLT),-1-0,Banks & Credit Unions,NaN,Finance,"Digital Product Manager, Digital Customer Onbo...",Hong Kong,Company - Private,3.8,some careers grow faster than others. if yo...,HK,Asia
1,Data Scientist,Harnham,51-200,Staffing & Outsourcing,Charing Cross,Business Services,Data Scientist media,United Kingdom,Company - Private,4.1,data scientist media london pou...,GB,Europe
2,Sales,Rentokil Initial,10000--1,Building & Personnel Services,Perai,Business Services,"Territory Manager (Outdoor Sales Consultant), ...",Malaysia,Company - Public,4.4,rentokil initial: rentokil initial is an ...,MY,Asia
3,Data Scientist,Derbysoft,51-200,IT Services,Barcelona,Information Technology,Data Scientist,Spain,Company - Private,3.8,job title: data scientist business unit:...,ES,Europe
4,Business Analyst,TrackTik,51-200,Computer Hardware & Software,Montréal,Information Technology,Business Analyst,Canada,Company - Private,4.2,about tracktik tracktik is a montreal bas...,CA,North America


In [58]:
stop = stopwords.words('english')
english['Cleaned Description'] = english['Job Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
english.head()

,Job Title,Employer Name,Employer Size,Industry,Location,Sector,Listing Job Title,Country,Company Type,Company Rating,Job Description,Code,Region,Cleaned Description
0,Product Manager,HSBC Global Technology (HSBC GLT),-1-0,Banks & Credit Unions,NaN,Finance,"Digital Product Manager, Digital Customer Onbo...",Hong Kong,Company - Private,3.8,some careers grow faster than others. if yo...,HK,Asia,careers grow faster others. rsquo looking care...
1,Data Scientist,Harnham,51-200,Staffing & Outsourcing,Charing Cross,Business Services,Data Scientist media,United Kingdom,Company - Private,4.1,data scientist media london pou...,GB,Europe,data scientist media london pound pound + bene...
2,Sales,Rentokil Initial,10000--1,Building & Personnel Services,Perai,Business Services,"Territory Manager (Outdoor Sales Consultant), ...",Malaysia,Company - Public,4.4,rentokil initial: rentokil initial is an ...,MY,Asia,rentokil initial: rentokil initial internation...
3,Data Scientist,Derbysoft,51-200,IT Services,Barcelona,Information Technology,Data Scientist,Spain,Company - Private,3.8,job title: data scientist business unit:...,ES,Europe,job title: data scientist business unit: marke...
4,Business Analyst,TrackTik,51-200,Computer Hardware & Software,Montréal,Information Technology,Business Analyst,Canada,Company - Private,4.2,about tracktik tracktik is a montreal bas...,CA,North America,tracktik tracktik montreal based saas scale co...


In [71]:
english.to_excel('./tableau_worldcloud.xlsx',engine='xlsxwriter')

In [91]:
# doc_frequency = Counter() # This will create a full counter of our terms. 
# [doc_frequency.update(item) for item in job_descriptions] # List comp
    
#     # Now we can just look at our final dict list inside doc_frequency
    
#     # Obtain our key terms and store them in a dict. These are the key data science skills we are looking for
    
#     prog_lang_dict = Counter({'R':doc_frequency['r'], 'Python':doc_frequency['python'],
#                     'Java':doc_frequency['java'], 'C++':doc_frequency['c++'],
#                     'Ruby':doc_frequency['ruby'],
#                     'Perl':doc_frequency['perl'], 'Matlab':doc_frequency['matlab'],
#                     'JavaScript':doc_frequency['javascript'], 'Scala': doc_frequency['scala']})
                      
#     analysis_tool_dict = Counter({'Excel':doc_frequency['excel'],  'Tableau':doc_frequency['tableau'],
#                         'D3.js':doc_frequency['d3.js'], 'SAS':doc_frequency['sas'],
#                         'SPSS':doc_frequency['spss'], 'D3':doc_frequency['d3']})  

#     hadoop_dict = Counter({'Hadoop':doc_frequency['hadoop'], 'MapReduce':doc_frequency['mapreduce'],
#                 'Spark':doc_frequency['spark'], 'Pig':doc_frequency['pig'],
#                 'Hive':doc_frequency['hive'], 'Shark':doc_frequency['shark'],
#                 'Oozie':doc_frequency['oozie'], 'ZooKeeper':doc_frequency['zookeeper'],
#                 'Flume':doc_frequency['flume'], 'Mahout':doc_frequency['mahout']})
                
#     database_dict = Counter({'SQL':doc_frequency['sql'], 'NoSQL':doc_frequency['nosql'],
#                     'HBase':doc_frequency['hbase'], 'Cassandra':doc_frequency['cassandra'],
#                     'MongoDB':doc_frequency['mongodb']})
                     
               
#     overall_total_skills = prog_lang_dict + analysis_tool_dict + hadoop_dict + database_dict # Combine our Counter objects
    
        
    
#     final_frame = pd.DataFrame(overall_total_skills.items(), columns = ['Term', 'NumPostings']) # Convert these terms to a 
#                                                                                                 # dataframe 
    
#     # Change the values to reflect a percentage of the postings 
    
#     final_frame.NumPostings = (final_frame.NumPostings)*100/len(job_descriptions) # Gives percentage of job postings 
#                                                                                     #  having that term 
    
#     # Sort the data for plotting purposes
    
#     final_frame.sort(columns = 'NumPostings', ascending = False, inplace = True)

,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,gaTrackerData.industry,gaTrackerData.industryId,gaTrackerData.jobTitle,gaTrackerData.location,gaTrackerData.locationId,gaTrackerData.locationType,...,map.location,overview.industry,overview.industryId,overview.revenue,overview.sector,overview.sectorId,overview.size,overview.type,rating.starRating,salary.salaries
0,-1,2183810,Modus Group,501-1000,Investment Banking & Asset Management,200053,Biogas Project Development Manager,Warsaw,3094484,CITY,...,Warsaw,Investment Banking & Asset Management,200053,Unknown / Non-Applicable,Finance,10010,501 to 1000 employees,Company - Private,-0.1,2483.0
1,-1,1043373,Immobiliare.it,201-500,NaN,0,"Senior Software Engineer (PHP, Elixir, Python)",Milano,2802090,CITY,...,Milan,NaN,0,£25 to £50 million (GBP),NaN,0,201 to 500 employees,Company - Private,2.4,2485.0
2,-1,2226886,Codup,1-50,NaN,0,Senior SQA Engineer,Karachi,3212296,CITY,...,Karāchi,NaN,0,Unknown / Non-Applicable,NaN,0,1 to 50 employees,Company - Private,1.5,2486.0
3,-1,851555,Markelytics Solutions,201-500,Consulting,200028,Research Manager,Bengaluru,2940587,CITY,...,Bengaluru,Consulting,200028,Unknown / Non-Applicable,Business Services,10006,201 to 500 employees,Company - Private,3.5,2487.0
4,-1,556626,Nisha Group,51-200,Staffing & Outsourcing,200032,B2B Product Manager,NaN,-1,CITY,...,Central,Staffing & Outsourcing,200032,Unknown / Non-Applicable,Business Services,10006,51 to 200 employees,Company - Private,3.5,2489.0


## Salary Dataset Cleaning


In [60]:
salary = pd.read_csv("./glassdoor_data/glassdoor_salary_salaries.csv")

In [61]:
salary.isnull().sum().sort_values(ascending = False)

salary.salaries.val.salaryPercentileMap.payPercentile50    219635
salary.salaries.val.salaryPercentileMap.payPercentile10     70406
salary.salaries.val.salaryPercentileMap.payPercentile90     70406
salary.salaries.val.jobTitle                                70399
index                                                       70372
salary.salaries.val.basePayCount                            70372
salary.salaries.val.payPeriod                               70372
salary.salaries.val.salaryType                              70372
id                                                              0
dtype: int64

In [63]:
salary = salary[salary['salary.salaries.val.salaryPercentileMap.payPercentile90'].notna()]
salary.head()

,id,index,salary.salaries.val.basePayCount,salary.salaries.val.jobTitle,salary.salaries.val.payPeriod,salary.salaries.val.salaryPercentileMap.payPercentile10,salary.salaries.val.salaryPercentileMap.payPercentile90,salary.salaries.val.salaryPercentileMap.payPercentile50,salary.salaries.val.salaryType
1,2,0.0,9.0,Advanced Consultant,ANNUAL,32257.68,41971.34,38713.25,employer
2,2,1.0,7.0,Consultant,ANNUAL,28855.30,74022.00,38602.20,employer
3,2,2.0,7.0,Software Engineer,ANNUAL,30377.86,38750.00,35032.53,employer
4,2,3.0,4.0,Project Manager,ANNUAL,38254.70,48675.67,45636.77,employer
5,2,4.0,3.0,Lead Consultant,ANNUAL,36724.92,49307.99,43344.02,employer


In [66]:
salary = salary[salary['salary.salaries.val.payPeriod']=='ANNUAL']
salary = salary[salary['salary.salaries.val.jobTitle'].notna()]
salary.head()
len(salary)

312699

In [72]:
def parse_title(title):
    title_lower = title.lower()
    
    title_list = ['data scientist', 'software engineer', 'data analyst', 'research scientist', 'business analyst',
                  'data engineer', 'statistician', 'dba', 'database engineer', 'machine learning engineer', 'applied scientist',
                 'business intelligence', 'product manager', 'sales']
    
    for val in title_list:
        if val in title_lower:
            return val.title()
   
    return title_lower.title()

salary['Cleaned Title'] = salary['salary.salaries.val.jobTitle'].apply(parse_title)
salary.head()

,id,index,salary.salaries.val.basePayCount,salary.salaries.val.jobTitle,salary.salaries.val.payPeriod,salary.salaries.val.salaryPercentileMap.payPercentile10,salary.salaries.val.salaryPercentileMap.payPercentile90,salary.salaries.val.salaryPercentileMap.payPercentile50,salary.salaries.val.salaryType,Cleaned Title
1,2,0.0,9.0,Advanced Consultant,ANNUAL,32257.68,41971.34,38713.25,employer,Advanced Consultant
2,2,1.0,7.0,Consultant,ANNUAL,28855.30,74022.00,38602.20,employer,Consultant
3,2,2.0,7.0,Software Engineer,ANNUAL,30377.86,38750.00,35032.53,employer,Software Engineer
4,2,3.0,4.0,Project Manager,ANNUAL,38254.70,48675.67,45636.77,employer,Project Manager
5,2,4.0,3.0,Lead Consultant,ANNUAL,36724.92,49307.99,43344.02,employer,Lead Consultant


In [73]:
# get only jobs of interest

title_list = ['data scientist', 'software engineer', 'data analyst', 'research scientist', 'business analyst',
              'data engineer', 'statistician', 'dba', 'database engineer', 'machine learning engineer', 'applied scientist',
             'business intelligence', 'product manager', 'sales']

filter_list =[]

for t in title_list:
    filter_list.append(t.title())
    
salary = salary[salary['salary.salaries.val.jobTitle'].isin(filter_list)]
len(salary)

23256

In [75]:
salary['Cleaned Title'].value_counts()

Software Engineer            10750
Business Analyst              4969
Product Manager               3496
Sales                         2355
Data Analyst                   863
Research Scientist             396
Data Scientist                 273
Data Engineer                   82
Business Intelligence           41
Machine Learning Engineer       14
Statistician                     9
Applied Scientist                4
Database Engineer                4
Name: Cleaned Title, dtype: int64

In [83]:
# get necessary data from jobs to map countries

jobs = df_g[['map.country', 'salary.salaries']]

jobs.rename(columns={'map.country': 'Country', 'salary.salaries': 'id'}, 
                            inplace=True)

jobs = pd.merge(jobs, country_codes, left_on='Country', right_on='Name', how='left')
jobs.head()

,Country,id,Name,Code
0,Poland,2483.0,Poland,PL
1,Italy,2485.0,Italy,IT
2,Pakistan,2486.0,Pakistan,PK
3,India,2487.0,India,IN
4,Israel,2489.0,Israel,IL


In [86]:
jobs.dropna(subset=['Code'], inplace=True)

In [87]:
jobs['Region'] = jobs['Code'].apply(pc.country_alpha2_to_continent_code)
jobs.drop(columns=['Name'], inplace=True)
def get_cont_name(code): 
    cont_conv_dict = {'AF': 'Africa',
                  'NA': 'North America',
                  'OC': 'Oceania',
                  'AN': 'Antarctica',
                  'AS': 'Asia',
                  'EU': 'Europe',
                  'SA': 'South America'}
    return cont_conv_dict[code] 

jobs['Region'] = jobs['Region'].apply(get_cont_name)
jobs.head()

,Country,id,Code,Region
0,Poland,2483.0,PL,Europe
1,Italy,2485.0,IT,Europe
2,Pakistan,2486.0,PK,Asia
3,India,2487.0,IN,Asia
4,Israel,2489.0,IL,Asia


In [88]:
salary_jobs = pd.merge(salary, jobs, left_on='id', right_on='id', how='left')
salary_jobs.head()

,id,index,salary.salaries.val.basePayCount,salary.salaries.val.jobTitle,salary.salaries.val.payPeriod,salary.salaries.val.salaryPercentileMap.payPercentile10,salary.salaries.val.salaryPercentileMap.payPercentile90,salary.salaries.val.salaryPercentileMap.payPercentile50,salary.salaries.val.salaryType,Cleaned Title,Country,Code,Region
0,2,2.0,7.0,Software Engineer,ANNUAL,30377.86,38750.00,35032.53,employer,Software Engineer,NaN,NaN,NaN
1,7,3.0,5.0,Business Analyst,ANNUAL,27177.39,48783.27,32975.96,employer,Business Analyst,NaN,NaN,NaN
2,12,3.0,2.0,Business Analyst,ANNUAL,27810.03,31743.78,NaN,employer,Business Analyst,China,CN,Asia
3,27,3.0,2.0,Product Manager,ANNUAL,54913.08,80426.90,NaN,employer,Product Manager,Sweden,SE,Europe
4,35,3.0,1.0,Business Analyst,ANNUAL,29532.63,31909.11,NaN,employer,Business Analyst,NaN,NaN,NaN


In [89]:
salary_jobs.dropna(subset=['Country'], inplace=True)
len(salary_jobs)

10606

In [90]:
salary_jobs.head()

,id,index,salary.salaries.val.basePayCount,salary.salaries.val.jobTitle,salary.salaries.val.payPeriod,salary.salaries.val.salaryPercentileMap.payPercentile10,salary.salaries.val.salaryPercentileMap.payPercentile90,salary.salaries.val.salaryPercentileMap.payPercentile50,salary.salaries.val.salaryType,Cleaned Title,Country,Code,Region
2,12,3.0,2.0,Business Analyst,ANNUAL,27810.03,31743.78,NaN,employer,Business Analyst,China,CN,Asia
3,27,3.0,2.0,Product Manager,ANNUAL,54913.08,80426.90,NaN,employer,Product Manager,Sweden,SE,Europe
8,53,3.0,1.0,Sales,ANNUAL,38325.99,41350.09,NaN,employer,Sales,Ireland,IE,Europe
16,102,0.0,22.0,Software Engineer,ANNUAL,26205.11,43448.66,33807.26,employer,Software Engineer,Singapore,SG,Asia
18,116,2.0,1.0,Product Manager,ANNUAL,47995.48,52268.21,NaN,employer,Product Manager,United Kingdom,GB,Europe


In [91]:
salary_jobs = salary_jobs[['id', 'salary.salaries.val.jobTitle', 'salary.salaries.val.salaryPercentileMap.payPercentile10', 
                           'salary.salaries.val.salaryPercentileMap.payPercentile90', 'Cleaned Title', 'Country', 'Code', 'Region']]

salary_jobs.rename(columns={'salary.salaries.val.jobTitle': 'Job Title', 
                          'salary.salaries.val.salaryPercentileMap.payPercentile10': 'PayPercentile10',
                          'salary.salaries.val.salaryPercentileMap.payPercentile90': 'PayPercentile90'}, 
                            inplace=True)
salary_jobs.head()

,id,Job Title,PayPercentile10,PayPercentile90,Cleaned Title,Country,Code,Region
2,12,Business Analyst,27810.03,31743.78,Business Analyst,China,CN,Asia
3,27,Product Manager,54913.08,80426.90,Product Manager,Sweden,SE,Europe
8,53,Sales,38325.99,41350.09,Sales,Ireland,IE,Europe
16,102,Software Engineer,26205.11,43448.66,Software Engineer,Singapore,SG,Asia
18,116,Product Manager,47995.48,52268.21,Product Manager,United Kingdom,GB,Europe


In [93]:
# save dataframe
salary_jobs.to_excel('./tableau_salaries.xlsx',engine='xlsxwriter', index=False)